In [ ]:
!./download_data.sh

### corrupted data cleanup

In [2]:
from PIL import Image
import glob
import shutil
import os
from keras.preprocessing.image import img_to_array, load_img
BROKEN = 'corrupted'

Using TensorFlow backend.


In [3]:
def move_to_broken(f):
    moveddir = os.path.dirname(os.path.join(BROKEN, f))
    os.path.isdir(moveddir) or os.makedirs(moveddir)
    shutil.copy(f, moveddir)
    os.remove(f)
    print("{} is broken, move it.".format(f))

In [ ]:
TARGET = "data"
TARGET_SIZE=(256, 256)

#        with Image.open(f) as dummy:
#            continue



os.makedirs(BROKEN, exist_ok=True)

for f in glob.iglob("{}/**/*.*".format(TARGET), recursive=True):
    try:
        _ = load_img(f, grayscale=False,
                           target_size=TARGET_SIZE)
    except:
        move_to_broken(f)

/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 3368026112 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 134479872 bytes but only got 0. Skipping tag 7
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4294901760 bytes but only got 0. Skipping tag 4
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.5/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expec

data/mountain/0126.jpeg is broken, move it.


In [ ]:
3

# Download data from urls

Now, get data from cloud storage is better. Below here is for initial data collection.

In [1]:
concepts = [
    "bay",
    "beach",
    "birds",
    "boeing",
    "buildings",
    "city",
    "clouds",
    "face",
    "f-16",
    "helicopter",
    "mountain",
    "sky",
    "ships",
    "sunset",
    "sunrise",
    "ocean"
]

In [2]:
def read_urls(cat):
    with open("urls/{}.txt".format(cat), encoding='utf8') as f:
        lines = f.readlines()
    return [line.rsplit('\n')[0] for line in lines]

In [3]:
from concurrent import futures
import urllib.request

    

In [4]:
accepted = {"jpeg", "png", "gif"}

def get_one(dirname, urlidx, urls):
    url = urls[urlidx]
    with urllib.request.urlopen(url) as response:
        ext = response.info().get_content_subtype()
        if not ext in accepted:
            return False
    urllib.request.urlretrieve(url, "{}/{:04d}.{}".format(dirname, urlidx, ext))
    return True

In [5]:
MAX_WORKERS = 10

In [6]:
def download_many_no_error_handle(urls, dirname):
    with futures.ThreadPoolExecutor(MAX_WORKERS) as executor:
        res = executor.map(lambda idx: get_one(dirname, idx, urls), range(len(urls)))
    _ = len(list(res))
    print("done")

In [7]:
import tqdm
from  http.client import RemoteDisconnected
from http.client import HTTPException
import ssl

def download_many(urls, dirname):
    to_do_map = {}
    fails = []
    with futures.ThreadPoolExecutor(MAX_WORKERS) as executor:
        for i in range(len(urls)):
            future = executor.submit(get_one,
                            dirname, i, urls)
            to_do_map[future] = i
        done_iter = futures.as_completed(to_do_map)
        
        done_iter = tqdm.tqdm(done_iter, total=len(urls))
        
        notify_err = lambda msg: None
        
        #notify_err = print

        
        for future in done_iter:
            idx = to_do_map[future]
            try:
                res = future.result()
                # print("deb")
                if not res:
                    notify_err("Unknown mime type: {}".format(urls[idx]))
                    fails.append(urls[idx])
            # make exception handling separately for debug purpose (now we can merge, but not yet)
            except (urllib.error.HTTPError, RemoteDisconnected, ssl.CertificateErr or, OSError, UnicodeEncodeError, urllib.error.URLError):
                notify_err("urllib retrieve rrorr: {}".format(urls[idx]))
                fails.append(urls[idx])
            except HTTPException:
                notify_err("HTTPException rrorr: {}".format(urls[idx]))
                fails.append(urls[idx])
                
    return fails

In [8]:
def download_one_category(cat):
    urls = read_urls(cat)
    dirname = "data/{}".format(cat)
    
    !mkdir -p $dirname
    fail_urls = download_many(urls, dirname)

In [9]:
list(map(download_one_category, concepts))

100%|██████████| 912/912 [02:20<00:00,  6.47it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Try and error

In [ ]:
list(map(download_one_category, concepts[5:]))

100%|█████████▉| 983/985 [01:39<00:00,  9.93it/s]

In [ ]:
download_one_category("birds")

In [17]:
download_one_category("beach")

100%|██████████| 1000/1000 [04:36<00:00,  3.61it/s]


In [9]:
cat = "birds"

In [10]:
urls = read_urls(cat)

In [11]:
dirname = "data/{}".format(cat)

!mkdir -p $dirname

In [12]:
# Need to trust this book

fail_urls = download_many(urls, dirname)

100%|██████████| 984/984 [03:13<00:00,  5.10it/s]


In [15]:
# [urls.index(f) for f in fail_urls]